# StuffDocumentsChain
- 문서 목록을 가져와서 모두 프롬프트에 포함시킨 후 LLM에 전달하는 Chain.
- 문서가 작고 적은 호출을 할 때 적합한 chain이다.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
print(os.environ["MODEL_ID"])

meta-llama/Meta-Llama-3-8B-Instruct


In [3]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert summarizer. Please summarize the following sentence.",
        ),
        (
            "user",
            "Please summarize the sentence according to the following request."
            "\nREQUEST:\n"
            "1. Summarize the main points in bullet points in Korean."
            "2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence."
            "3. Use various emojis to make the summary more interesting."
            "\n\nCONTEXT: {context}\n\nSUMMARY:",
        ),
    ]
)
prompt

ChatPromptTemplate(input_variables=['context'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an expert summarizer. Please summarize the following sentence.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='Please summarize the sentence according to the following request.\nREQUEST:\n1. Summarize the main points in bullet points in Korean.2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.3. Use various emojis to make the summary more interesting.\n\nCONTEXT: {context}\n\nSUMMARY:'))])

In [9]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/news.txt")
docs = loader.load()
print(type(docs[0]))
print(f"문서의 수: {len(docs)}\n")
print("[메타데이터]\n")
print(docs[0].metadata)
print("\n========= [앞부분] 미리보기 =========\n")
print(docs[0].page_content[:500])

<class 'langchain_core.documents.base.Document'>
문서의 수: 1

[메타데이터]

{'source': 'data/news.txt'}

========= [앞부분] 미리보기 =========

제목: 
AI2, 상업 활용까지 자유로운 '진짜' 오픈 소스 LLM '올모' 출시

내용:
앨런AI연구소(AI2)가 완전한 오픈 소스 대형언어모델(LLM) '올모(OLMo)’를 출시했다. 데이터 수집, 학습, 배포의 전 과정을 투명하게 공개한 데다 상업적 사용까지 허용한 진정한 의미의 오픈 소스 LLM이라는 평가다.
벤처비트는 1일(현지시간) 비영리 민간 AI 연구기관인 AI2가 ‘최초의 진정한 오픈 소스 LLM 및 프레임워크’라고 소개한 ‘올모’를 출시했다고 보도했다. 
이에 따르면 올모는 모델 코드와 모델 가중치뿐만 아니라 훈련 코드, 훈련 데이터, 관련 툴킷 및 평가 툴킷도 제공한다. 이를 통해 모델이 어떻게 구축되었는지 심층적으로 분석, LLM의 작동 방식과 응답을 생성하는 원리를 더 잘 이해할 수 있다. 
올모 프레임워크는 70억 매개변수의 ‘올모 7B’ 등 4가지 변형 모델과 10억 매개변수의 ‘올모 1B’ 모델을 제공한다. 모델들은 훈련 데이터를 생성하는 코드를 포함해 


In [12]:
from langchain.callbacks.base import BaseCallbackHandler
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain_community.llms import HuggingFaceEndpoint

class MyCallbackHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(f"{token}", end="", flush=True)


llm = HuggingFaceEndpoint(
    repo_id=os.environ["MODEL_ID"], 
    max_new_tokens=1024,
    temperature=0.1,
    streaming=True,
    callbacks=[MyCallbackHandler()],
    huggingfacehub_api_token=os.environ["HF_API_KEY"],
)
model = ChatHuggingFace(llm=llm)

chain = create_stuff_documents_chain(model, prompt)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dudaji/.cache/huggingface/token
Login successful


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
print(chain.invoke({"context": docs}))

Here is the summary in Korean, with each point starting with an emoji:

• 🎉 올모(OLMo)는 앨런AI연구소(AI2)가 출시한 완전한 오픈 소스 대형언어모델(LLM)입니다.
• 💡 올모는 모델 코드, 모델 가중치, 훈련 코드, 훈련 데이터, 관련 툴킷, 평가 툴킷 등을 제공하여 모델의 구축 과정을 심층적으로 분석할 수 있습니다.
• 📊 올모 프레임워크에는 4가지 변형 모델(올모 7B, 올모 1B 등)이 포함되어 있습니다.
• 💻 프레임워크에는 추론 코드, 훈련 지표, 훈련 로그, 모델 가중치, 평가 제품군 등이 포함되어 있습니다.
• 📈 올모는 상업적 활용에 제한이 없습니다. 아파치 2.0 라이선스에 따라 사용할 수 있습니다.
• 🤔 AI2는 올모를 통해 연구자들이 안전하고 신뢰할 수 있는 차세대 시스템을 구축하는 데 중요한 LLM 과학을 연구할 수 있게 될 것이라고 말합니다.
• 📊 성능 면에서는 올모가 상업용 제품과 동등한 성능을 보여주는 것으로 나타났습니다.
• 🚫 다만 비영어권 언어에 대한 낮은 품질과 약한 코드 생성 기능과 같은 제약 사항이 있습니다.
• 💪 AI2는 올모를 계속해서 향상할 것이라고 말합니다.

Note: The summary is based on the provided text and may not include all the details.<|eot_id|>
